In [1]:
import zipfile
import tempfile
from pathlib import Path
import xarray as xr
import pandas as pd

# -------------------------------
# CONFIG
# -------------------------------
for i in range(1,6):
    file_path = Path(f"era5_data/era5_2025_0{i}.nc")  # your downloaded ERA5 file
    output_csv = f"era5_full_dataset_0{i}.csv"

    # -------------------------------
    # LOAD DATASET (handle zip or nc)
    # -------------------------------
    if not file_path.exists():
        raise FileNotFoundError(file_path)

    with open(file_path, 'rb') as f:
        sig = f.read(4)

    if sig[:2] == b'PK':
        print('File is a zip archive. Extracting .nc...')
        with zipfile.ZipFile(file_path, 'r') as z:
            nc_files = [n for n in z.namelist() if n.lower().endswith('.nc')]
            if not nc_files:
                raise RuntimeError('Zip archive does not contain any .nc files')
            member = nc_files[0]
            with tempfile.NamedTemporaryFile(suffix='.nc', delete=False) as tmp:
                tmp.write(z.read(member))
                tmp_path = Path(tmp.name)
            ds = xr.open_dataset(tmp_path)
    else:
        print('File is not a zip; opening directly')
        ds = xr.open_dataset(file_path)

    print('Dataset loaded successfully!')

    # -------------------------------
    # STACK LAT/LON and convert to DataFrame
    # -------------------------------
    print('Converting dataset to DataFrame...')

    # Flatten lat/lon into single "points" dimension
    df = ds.to_dataframe().reset_index()

    # Convert temperature from K to °C for common variables
    for var in ['t2m']:
        if var in df.columns:
            df[var] = df[var] - 273.15

    # -------------------------------
    # SAVE TO CSV
    # -------------------------------
    print(f'Saving full dataset to CSV: {output_csv} ...')
    df.to_csv(output_csv, index=False)
    print('Done!')


File is a zip archive. Extracting .nc...
Dataset loaded successfully!
Converting dataset to DataFrame...
Saving full dataset to CSV: era5_full_dataset_01.csv ...
Done!
File is a zip archive. Extracting .nc...
Dataset loaded successfully!
Converting dataset to DataFrame...
Saving full dataset to CSV: era5_full_dataset_02.csv ...
Done!
File is a zip archive. Extracting .nc...
Dataset loaded successfully!
Converting dataset to DataFrame...
Saving full dataset to CSV: era5_full_dataset_03.csv ...
Done!
File is a zip archive. Extracting .nc...
Dataset loaded successfully!
Converting dataset to DataFrame...
Saving full dataset to CSV: era5_full_dataset_04.csv ...
Done!
File is a zip archive. Extracting .nc...
Dataset loaded successfully!
Converting dataset to DataFrame...
Saving full dataset to CSV: era5_full_dataset_05.csv ...
Done!
